In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,chi2

In [2]:
X_test=pd.read_csv("/Users/MacAir/Documents/DataScience/Datasets/home-data-for-ml-course/test.csv",index_col='Id')
traindata=pd.read_csv("/Users/MacAir/Documents/DataScience/Datasets/home-data-for-ml-course/train.csv",index_col='Id')

In [3]:
y=traindata["SalePrice"]
X=traindata.drop("SalePrice", axis=1)

In [4]:
X.shape

(1460, 79)

In [5]:
low_cardinality_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and X[cname].dtype == "object"]
numeric_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
print(len(numeric_cols))
print(len(low_cardinality_cols))
my_cols = numeric_cols+low_cardinality_cols
X=X[my_cols]
X_test=X_test[my_cols]

36
40


In [6]:
X_test.shape

(1459, 76)

In [7]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2,random_state=0)
bad_cols=[col for col in low_cardinality_cols if (bool(set(X_valid[col].unique()).difference(set(X_train[col].unique()))))]
X_train.drop(columns=bad_cols,inplace=True)
X_valid.drop(columns=bad_cols,inplace=True)

In [8]:
#Imputer
trf1=ColumnTransformer([
    ("Numerical Imputer",SimpleImputer(strategy='median'),slice(0,36)),
    ("Text Imputer",SimpleImputer(strategy='most_frequent'),slice(36,76))
],remainder='passthrough')

In [9]:
#Encoder column
trf2=ColumnTransformer([
    ("Ordinal Encoding",OrdinalEncoder(),slice(36,76))
],remainder='passthrough')

In [10]:
#Scaler
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,76))
])

In [11]:
# train the model
trf4 = XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=8)

In [12]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4)
])

In [13]:
from sklearn import set_config
set_config(display='diagram')
pipe.fit(X_train,y_train)#trf4__early_stopping_rounds=8, 
           # trf4__eval_set=[(X_valid, y_valid)], 
           #  trf4__verbose=False)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Numerical Imputer',
                                                  SimpleImputer(strategy='median'),
                                                  slice(0, 36, None)),
                                                 ('Text Imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  slice(36, 76, None))])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Ordinal Encoding',
                                                  OrdinalEncoder(),
                                                  slice(36, 76, None))])),...
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.05, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=8, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              reg_lambda=1, ...))])

In [14]:
y_pred = pipe.predict(X_valid)
y_pred

array([219851.3  , 155607.06 , 108246.59 , 217298.   ,  89014.555,
       104673.95 , 262147.78 , 127267.664, 643343.7  , 153219.72 ,
       194195.78 , 143507.27 , 228586.1  , 124645.92 , 122760.55 ,
       146808.36 , 217226.23 , 123064.35 , 136755.83 , 202734.44 ,
       123141.02 , 141641.58 , 114191.62 , 158099.72 , 184000.39 ,
       214187.2  , 170157.95 ,  82406.09 , 319426.16 , 112796.45 ,
       131655.44 , 198381.98 , 141991.7  , 280757.34 , 328080.84 ,
       186670.97 , 292156.12 , 121475.58 , 222474.88 , 305529.6  ,
       232969.77 , 125222.08 , 184755.98 , 288385.5  , 365759.7  ,
       139970.42 , 132063.17 , 126031.62 , 165100.03 ,  99156.12 ,
       382575.94 , 144120.48 , 167151.14 ,  75875.016, 239970.9  ,
       113505.445, 137754.66 , 246778.73 , 137330.81 ,  99901.66 ,
       134423.11 , 129856.984, 142473.55 , 159033.39 , 208549.45 ,
       140622.86 , 126971.8  , 218732.03 , 120454.32 , 188669.69 ,
       175854.73 , 121889.22 ,  88240.51 , 229325.1  ,  92234.

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred,y_valid)

16935.464415667808

In [18]:
params = {
    'trf4__early_stopping_rounds':[8],
    'trf4__eval_set':[(X_valid, y_valid)],
    'trf4__verbose':[False]
}

In [19]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=8, scoring='accuracy')
grid.fit(X_train, y_train)

[09:12:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:627: 
Parameters: { "eval_set", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:12:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:627: 
Parameters: { "eval_set", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:12:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.mac

/Users/MacAir/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
8 fits failed out of a total of 8.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/MacAir/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/MacAir/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/MacAir/miniforge3/envs/tensorflow/lib/python3.9/site-packages/xgboost

[09:12:33] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/learner.cc:627: 
Parameters: { "eval_set", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




AssertionError: Must have at least 1 validation dataset for early stopping.